In [2]:
import pandas as pd

In [3]:
movies = pd.read_csv('/home/antoine/jul24_cmlops_reco_film/ml/data/processed/processed_movies.csv')

movies.head()

,movieId,title,genres,year
0,1,Toy Story,"['Adventure', 'Animation', 'Children', 'Comedy...",1995
1,2,Jumanji,"['Adventure', 'Children', 'Fantasy']",1995
2,3,Grumpier Old Men,"['Comedy', 'Romance']",1995
3,4,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",1995
4,5,Father of the Bride Part II,['Comedy'],1995


In [4]:
ratings = pd.read_csv('/home/antoine/jul24_cmlops_reco_film/ml/data/processed/processed_ratings.csv')

ratings.head()

,userId,movieId,rating,timestamp,bayesian_mean
0,1,2,3.5,1112486027,3.209414
1,1,29,3.5,1112484676,3.886141
2,1,32,3.5,1112484819,3.885546
3,1,47,3.5,1112484727,4.037850
4,1,50,3.5,1112484580,4.315561


In [5]:
links = pd.read_csv('/home/antoine/jul24_cmlops_reco_film/ml/data/processed/processed_links.csv')

links.head()

,movieId,imdbId,tmdbId
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862


In [6]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from dotenv import load_dotenv
import psycopg2
import csv
import os

base_dir = '/home/antoine/jul24_cmlops_reco_film/ml/src/data'
# Construire le chemin vers le répertoire contenant les données traitées
env_dir = os.path.join(base_dir, '..', '..', '..', 'postgres')

# Charger les variables d'environnement depuis le fichier .env
load_dotenv(env_dir)

# Configuration de la base de données
DB_NAME = os.getenv('POSTGRES_DB')
DB_USER = os.getenv('POSTGRES_USER')
DB_PASSWORD = os.getenv('POSTGRES_PASSWORD')
DB_HOST = os.getenv('POSTGRES_HOST')
DB_PORT = os.getenv('POSTGRES_PORT')


In [8]:
print(DB_PASSWORD)

datascientest
